In [1]:
import csv
import math
import matplotlib.pyplot as plt
from io import StringIO
from ipywidgets import Button, VBox, Output, Textarea, HTML, HBox, Dropdown
from IPython.display import display, FileLink

EPS = 1e-9
MPS_TO_KMH = 3.6
MPS_TO_MPH = 2.23694
M_TO_KM = 1/1000
M_TO_MI = 1/1609.344

def compute_scale_factors(ref_lat_deg):
    ref_lat_rad = math.radians(ref_lat_deg)
    Mphi = 111132 - 559 * math.cos(2 * ref_lat_rad) + 1.175 * math.cos(4 * ref_lat_rad)
    Mlambda = 111320 * math.cos(ref_lat_rad) - 94 * math.cos(3 * ref_lat_rad)
    return Mphi, Mlambda

def process_csv(input_data, lat_col, lon_col, time_col):
    f = StringIO(input_data)
    reader = csv.DictReader(f)
    headers = [h.strip() for h in reader.fieldnames]
    rows = [{k.strip(): v for k, v in r.items()} for r in reader]

    if len(rows) < 2:
        print("Not enough rows to compute speeds.")
        return

    ref_lat = float(rows[0][lat_col])
    Mphi, Mlambda = compute_scale_factors(ref_lat)

    for row in rows:
        for col in [
            "relative_time",
            "segment_distance_m", "segment_distance_km", "segment_distance_mi",
            "cumulative_distance_m", "cumulative_distance_km", "cumulative_distance_mi",
            "speed_mps", "speed_kmh", "speed_mph"
        ]:
            row[col] = ""

    t0 = float(rows[0][time_col])
    cumulative_distance_m = 0
    times, speeds_mph = [], []

    for i in range(len(rows)):
        t = float(rows[i][time_col])
        relative_t = t - t0
        rows[i]["relative_time"] = f"{relative_t:.2f}"

        if i == 0:
            for key in [
                "segment_distance_m", "segment_distance_km", "segment_distance_mi",
                "cumulative_distance_m", "cumulative_distance_km", "cumulative_distance_mi",
                "speed_mps", "speed_kmh", "speed_mph"
            ]:
                rows[i][key] = "0.00"
            continue

        lat1 = float(rows[i-1][lat_col])
        lon1 = float(rows[i-1][lon_col])
        t1 = float(rows[i-1][time_col])

        lat2 = float(rows[i][lat_col])
        lon2 = float(rows[i][lon_col])
        t2 = float(rows[i][time_col])

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        if abs(dlat) < EPS and abs(dlon) < EPS:
            segment_distance_m = 0
        else:
            dx = dlon * Mlambda
            dy = dlat * Mphi
            segment_distance_m = math.sqrt(dx*dx + dy*dy)

        cumulative_distance_m += segment_distance_m
        dt = t2 - t1

        if dt > 0:
            speed_mps = segment_distance_m / dt
            speed_kmh = speed_mps * MPS_TO_KMH
            speed_mph = speed_mps * MPS_TO_MPH
        else:
            speed_mps = speed_kmh = speed_mph = float('nan')

        rows[i]["segment_distance_m"] = f"{segment_distance_m:.2f}"
        rows[i]["segment_distance_km"] = f"{segment_distance_m*M_TO_KM:.3f}"
        rows[i]["segment_distance_mi"] = f"{segment_distance_m*M_TO_MI:.3f}"
        rows[i]["cumulative_distance_m"] = f"{cumulative_distance_m:.2f}"
        rows[i]["cumulative_distance_km"] = f"{cumulative_distance_m*M_TO_KM:.3f}"
        rows[i]["cumulative_distance_mi"] = f"{cumulative_distance_m*M_TO_MI:.3f}"
        rows[i]["speed_mps"] = f"{speed_mps:.2f}"
        rows[i]["speed_kmh"] = f"{speed_kmh:.2f}"
        rows[i]["speed_mph"] = f"{speed_mph:.2f}"

        times.append(relative_t)
        speeds_mph.append(speed_mph)

    new_headers = headers + [
        "relative_time",
        "segment_distance_m", "segment_distance_km", "segment_distance_mi",
        "cumulative_distance_m", "cumulative_distance_km", "cumulative_distance_mi",
        "speed_mps", "speed_kmh", "speed_mph"
    ]

    with open("results.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=new_headers)
        writer.writeheader()
        writer.writerows(rows)

    print("✅ Results saved to results.csv")
    display(FileLink("results.csv"))

    plt.figure(figsize=(10,5))
    plt.plot(times, speeds_mph, marker='o')
    plt.xlabel('Relative Time (s)')
    plt.ylabel('Speed (mph)')
    plt.grid(True)
    plt.show()

def run_with_ui():
    output = Output()
    input_box = Textarea(
        placeholder="Paste your CSV data here...",
        layout={'width':'100%', 'height':'200px'}
    )

    process_button = Button(description="🚀 Process CSV", button_style='success')
    example_button = Button(description="📄 Load Example CSV", button_style='info')

    example_csv = """time,lat,lon
0,40.0000,-105.0000
1,40.0005,-105.0003
2,40.0011,-105.0007
3,40.0016,-105.0010
"""

    def load_example(b):
        input_box.value = example_csv
        with output:
            output.clear_output()
            print("📄 Example CSV loaded.")

    example_button.on_click(load_example)

    def start_processing(b):
        with output:
            output.clear_output()
            text = input_box.value.strip()
            if not text:
                print("⚠️ Paste CSV data OR click Load Example CSV.")
                return

            f = StringIO(text)
            reader = csv.DictReader(f)
            headers = reader.fieldnames

            if not headers:
                print("❌ Could not read CSV header row.")
                return

            lat_dd = Dropdown(options=headers, description='Latitude:')
            lon_dd = Dropdown(options=headers, description='Longitude:')
            time_dd = Dropdown(options=headers, description='Timestamp:')

            go_button = Button(description="✅ Process", button_style='success')

            def really_process(b2):
                with output:
                    output.clear_output()
                    try:
                        process_csv(text, lat_dd.value, lon_dd.value, time_dd.value)
                        print("Done.")
                    except Exception as e:
                        print(f"❌ Error: {e}")

            go_button.on_click(really_process)

            display(VBox([
                HTML('Select Columns:'),
                HBox([lat_dd, lon_dd, time_dd]),
                go_button
            ]))

    process_button.on_click(start_processing)

    display(VBox([
        HBox([example_button, process_button]),
        input_box,
        output
    ]))

run_with_ui()